# This notebook is utilize for parameter tuning, using Keras RandomSearchCV and GridSearchCV

### Function to get the CNN module to be wrapped with Scikit_learn wrappers

In [ ]:
def get_CNN_model(size = 64,target_type='vectorOrientations',optimizer='adam',dropout_rate1=0.0,dropout_rate2=0.0, fc1 = 512, fc2 = 64,padding1 = 'valid',padding2='valid'):
    
    if target_type == 'quaternions':
        output_layer = 4
    else:
        output_layer = 6
    
    model = Sequential()
    
    # Set up Convolutional layers
    model.add(Conv2D(96, (11, 11),padding=padding1, input_shape=(size, size, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(384, (5, 5), padding = padding2))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(256, (5, 5)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Set up Fully Connected layers
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(fc1))
    model.add(Activation('relu'))
    model.add(Dropout(rate=dropout_rate1))
    model.add(Dense(fc2))
    model.add(Activation('relu'))
    model.add(Dropout(rate=dropout_rate2))
    model.add(Dense(output_layer))
    model.add(Activation('linear')) #due to this is a regression problem
    
    # Compile the model
    model.compile(loss=customized_mse,
    optimizer=optimizer)#,
    #metrics=['accuracy'])

    return model



### Customized_mse function
- modification of standard MSE, adding scaling of the vedctor to the unit length
- take input of true test target and predictions
- return the mse

In [ ]:
def customized_mse(y_true,y_pred):
    vector_1 = y_pred[:,:3]
    vector_2 = y_pred[:,3:]

    # To normalize the predictions vectors to have length of 1
    normalized_vector_1 = tf.divide(vector_1,tf.sqrt(tf.reduce_sum(tf.square(vector_1),1,keepdims=True)))
    normalized_vector_2 = tf.divide(vector_2,tf.sqrt(tf.reduce_sum(tf.square(vector_2),1,keepdims=True)))
    normalized_y_pred = tf.concat([normalized_vector_1, normalized_vector_2], axis=1)
    return tf.reduce_mean(tf.square(tf.subtract(normalized_y_pred,y_true)))

### Function for plotting

In [ ]:
def plot_learning_curves(history,optimizer):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.xlabel("Epoch",fontsize=12)
    plt.ylabel("Loss",fontsize=12)
    plt.title("Optimizer: "+optimizer,fontsize=12,fontweight="bold")
    plt.show()

### Function for generating train and validation data

In [ ]:
num_images = 1500
image_size = 64
target_type = 'vectorOrientations'

rendered_images = get_images(num_images,image_size)
    
# convert into x_train, xVal, yTrain and yVal
xTrain, xVal, yTrain, yVal= get_train_val(rendered_images,target_type)
    
# Reshape the train and validation set
len_train = len(xTrain)
len_val = len(xVal)
xTrain = np.array(xTrain).reshape(len_train,image_size,image_size,3)
xVal = np.array(xVal).reshape(len_val,image_size,image_size,3)

### Set Parameters and scikit_learn wrapper

In [ ]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(get_CNN_model)

param_distribs = {
    "optimizer": ['adam','adadelta','rmsprop','sgd','adagrad','nadam','adamax'],
    "dropout_rate1": [0,0.2,0.3,0.5],
    "dropout_rate2": [0,0.2,0.3,0.5],
    "epochs": [30,50,100],
    "batch_size":[20,30,40,64,128],
    "fc1": [512,1024,256],
    "fc2": [64,256,32],
    "padding1":['valid','same'],
    "padding2":['valid','same']
}


## Use RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
random_search_cv = RandomizedSearchCV(estimator = keras_reg, param_distributions = param_distribs, n_jobs=1, n_iter=5, cv=3, verbose=2)
random_search_cv.fit(xTrain, yTrain,
                  validation_data=(xVal, yVal),
                  callbacks=[keras.callbacks.EarlyStopping(patience=5)])

In [ ]:
random_search_cv.best_params_

In [ ]:
random_search_cv.best_score_

In [ ]:
random_search_cv.best_estimator_

In [ ]:
#Getting the images for testing (on later stage should get images from Nil's module)
image_test = get_images(200,64)
x_test = np.array(image_test['images'])
y_test = np.array([np.concatenate(x) for x in image_test['vectorOrientations']])

In [ ]:
random_search_cv.score(x_test, y_test)

In [ ]:
model_rnd = random_search_cv.best_estimator_.model
model_rnd.evaluate(x_test, y_test)

In [ ]:
model_rnd.save("renny/tuned_model_rnd_1_12April.h5")

## Use GridSearchCV

In [ ]:
xTrain.shape

In [ ]:
grid_search_cv = GridSearchCV(estimator = keras_reg, param_grid = param_distribs, verbose=2)
grid_search_cv.fit(xTrain, yTrain,
                  validation_data=(xVal, yVal),
                  callbacks=[keras.callbacks.EarlyStopping(patience=5)])

In [ ]:
grid_search_cv.best_params_

In [ ]:
grid_search_cv.best_score_

In [ ]:
grid_search_cv.best_estimator_

## Search of Learning rates parameter

In [ ]:
import matplotlib.pyplot as plt
optimizers = ['adam','adadelta','rmsprop','sgd','adagrad','nadam','adamax']
histories = []
for optimizer in optimizers:
    model = get_CNN_model(optimizer= optimizer)
    callbacks = [keras.callbacks.EarlyStopping(patience=10)]
    history = model.fit(xTrain, yTrain,
                        validation_data=(xVal, yVal), epochs=50,batch_size=50,
                        callbacks=callbacks)
    histories.append(history)

In [ ]:
for optimizer, history in zip(optimizers, histories):
    plot_learning_curves(history,optimizer)


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10,40))

n_optimizers = len(optimizers)


ax = fig.add_subplot(n_optimizers,1,1)
ax.plot(range(1,len(histories[0].history['loss'])+1), histories[0].history['val_loss'],label='val_loss')
ax.plot(range(1,len(histories[0].history['loss'])+1), histories[0].history['loss'], label='loss')
plt.title('Optimizer = '+optimizers[0],fontweight="bold")
ax.set_ylabel('Loss')
ax.set_xlabel('epoch')
ax.grid()
plt.gca().set_ylim(0, 1)
ax.legend()
    
ax1 = fig.add_subplot(n_optimizers,1,2)
ax1.plot(range(1,len(histories[1].history['loss'])+1), histories[1].history['val_loss'],label='val_loss')
ax1.plot(range(1,len(histories[1].history['loss'])+1), histories[1].history['loss'], label='loss')
plt.title('Optimizer = '+optimizers[1],fontweight="bold")
ax1.set_ylabel('Loss')
ax1.set_xlabel('epoch')
ax1.grid(True)
plt.gca().set_ylim(0, 1)
ax1.legend()
    
ax2 = fig.add_subplot(n_optimizers,1,3)
ax2.plot(range(1,len(histories[2].history['loss'])+1), histories[2].history['val_loss'],label='val_loss')
ax2.plot(range(1,len(histories[2].history['loss'])+1), histories[2].history['loss'], label='loss')
plt.title('Optimizer = '+optimizers[2],fontweight="bold")
ax2.set_ylabel('Loss')
ax2.set_xlabel('epoch')
ax2.grid(True)
plt.gca().set_ylim(0, 1)
ax2.legend()
        
ax3 = fig.add_subplot(n_optimizers,1,4)
ax3.plot(range(1,len(histories[3].history['loss'])+1), histories[3].history['val_loss'],label='val_loss')
ax3.plot(range(1,len(histories[3].history['loss'])+1), histories[3].history['loss'], label='loss')
plt.title('Optimizer = '+optimizers[0],fontweight="bold")
ax3.set_ylabel('Loss')
ax3.set_xlabel('epoch')
ax3.grid(True)
plt.gca().set_ylim(0, 1)
ax3.legend()
        
ax4 = fig.add_subplot(n_optimizers,1,5)
ax4.plot(range(1,len(histories[4].history['loss'])+1), histories[4].history['val_loss'],label='val_loss')
ax4.plot(range(1,len(histories[4].history['loss'])+1), histories[4].history['loss'], label='loss')
plt.title('Optimizer = '+optimizers[4],fontweight="bold")
ax4.set_ylabel('Loss')
ax4.set_xlabel('epoch')
ax4.grid(True)
plt.gca().set_ylim(0, 1)
ax4.legend()
        
ax5 = fig.add_subplot(n_optimizers,1,6)
ax5.plot(range(1,len(histories[5].history['loss'])+1), histories[5].history['val_loss'],label='val_loss')
ax5.plot(range(1,len(histories[5].history['loss'])+1), histories[5].history['loss'], label='loss')
plt.title('Optimizer = '+optimizers[5],fontweight="bold")
ax5.set_ylabel('Loss')
ax5.set_xlabel('epoch')
ax5.grid(True)
plt.gca().set_ylim(0, 1)
ax5.legend()
        
ax6 = fig.add_subplot(n_optimizers,1,7)
ax6.plot(range(1,len(histories[6].history['loss'])+1), histories[6].history['val_loss'],label='val_loss')
ax6.plot(range(1,len(histories[6].history['loss'])+1), histories[6].history['loss'], label='loss')
plt.title('Optimizer = '+optimizers[6],fontweight="bold")
ax6.set_ylabel('Loss')
ax6.set_xlabel('epoch')
ax6.grid(True)

ax6.legend()

plt.show()  
plt.savefig('learning_curve.png')  
